# Correccion de Examen 2 

### Jesús Emmanuel Flores Cortés 

## Examen Teorico

### 1.- Explica el modelo de regresión logística para clasificación. ¿Cómo se determina el umbral de decisión?

La regresión logística sirve para clasificar observaciones en dos clases. Calcula la probabilidad de pertenecer a una clase usando una función sigmoide que transforma una combinación lineal de las variables en un valor entre 0 y 1.
El umbral de decisión es el punto que separa las clases, normalmente 0.5. Si la probabilidad es mayor al umbral, se clasifica en una clase; si es menor, en la otra. El umbral puede ajustarse dependiendo del tipo de error que se quiera minimizar.

### 2.-Explica la intuición de la máquina de soporte vectorial para clasificación. ¿Cómo se determina qué modelo es mejor? ¿Cuál es la mayor diferencia que tiene contra un modelo de regresión logística?

La SVM busca una frontera que divida las clases de forma que la distancia entre los puntos más cercanos de cada clase y esa frontera sea máxima. Esa distancia se llama margen.
Para decidir qué modelo es mejor, se comparan métricas como precisión, recall, F1 o exactitud en validación cruzada.
La principal diferencia con la regresión logística es que la SVM no calcula probabilidades, sino que se enfoca en encontrar la frontera que mejor separa las clases.

### 3.- ¿Cuáles son los componentes principales en un MLP para clasificación? Dibuja un ejemplo y señaliza.

Un MLP está compuesto por:

Una capa de entrada que recibe las variables

Una o más capas ocultas donde las neuronas combinan las entradas aplicando pesos y funciones de activación

Una capa de salida que entrega la predicción final.
Cada conexión tiene un peso que se ajusta durante el entrenamiento para minimizar el error

### 4.- ¿Cuál es el procedimiento a seguir cuando los datos no son linealmente separables en una SVC?

Cuando los datos no son linealmente separables, se usa un kernel, que transforma los datos a un espacio de mayor dimensión donde sí pueden separarse. Algunos kernels comunes son lineal, polinomial y RBF.

### 5.- Describe qué es un hiperparámetro. ¿Por qué es importante ajustarlos? Da dos ejemplos de hiperparámetros.


Los hiperparámetros son valores que definen el comportamiento del modelo antes del entrenamiento. No se aprenden de los datos, se eligen y ajustan para mejorar el rendimiento.
Ejemplos: el parámetro C en una SVM (controla la penalización del error) y la tasa de aprendizaje en un MLP.

### 6.- Dibuja un diagrama de flujo para describir el proceso de optimización Bayesiana.

Definir el espacio de hiperparámetros

Elegir un modelo probabilístico inicial 

Probar un conjunto de valores de hiperparámetros y medir el desempeño

Actualizar el modelo probabilístico con los resultados

Seleccionar los siguientes hiperparámetros a probar según la probabilidad de mejorar el resultado

Repetir hasta encontrar los mejores valores

### 7.- ¿Qué es la curva ROC y cómo se usa para evaluar el desempeño de un modelo?

La curva ROC muestra la relación entre la tasa de verdaderos positivos (TPR) y la tasa de falsos positivos (FPR) al variar el umbral de decisión.
Se usa para evaluar el desempeño general de un modelo de clasificación. Un modelo es mejor mientras más se acerque la curva a la esquina superior izquierda. El área bajo la curva (AUC) resume su calidad.

### 8.- Describe un espacio de Hilbert.

Es un espacio matemático donde se pueden aplicar operaciones como suma, producto interno y medición de distancia. Es importante en aprendizaje automático porque permite representar datos en dimensiones altas y aplicar kernels de forma eficiente.

### 9.- ¿Qué significa que una función de costo sea convexa? ¿Qué beneficios hay de que un modelo tenga una función de costo convexa?

Una función de costo convexa tiene una sola solución mínima global. Esto significa que el proceso de optimización no se queda atrapado en mínimos locales. Tener una función convexa garantiza que el modelo converge hacia la mejor solución posible.

### 10.- Piensa en los 3 modelos aprendidos en este parcial: ¿En qué situaciones usarías cada uno y por qué?

Regresión logística: cuando las clases son lineales y se buscan probabilidades interpretables.

SVM: cuando los datos no son lineales y se necesita una separación clara entre clases.

MLP: cuando la relación entre variables es compleja o no lineal, y se dispone de más datos y poder de cómputo.

# Examen Practico

# Pregunta 1

In [59]:
import warnings
import numpy as np
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RBF
warnings.filterwarnings("ignore")

In [60]:
def funcion_objetivo(X):
    x, y, z = X[:, 0], X[:, 1], X[:, 2]
    
    term_x = (6*x - 2)**2 * np.sin(12*x - 4)
    term_y = (6*y - 2)**2 * np.cos(12*y - 4)
    term_z = (6*z - 2)**2 * np.sin(12*z - 4)
    
    return term_x + term_y + term_z

In [61]:
def expected_improvement(X_candidatos, gpr, f_mejor):

    mu, sigma = gpr.predict(X_candidatos, return_std=True)

    with np.errstate(divide='ignore'):
        Z = (f_mejor - mu) / sigma
        
        ei = sigma * (Z * norm.cdf(Z) + norm.pdf(Z))
        ei[sigma == 0.0] = 0.0
        
    return ei

In [62]:
def proximo_punto_optimo(gpr, f_mejor, bounds, n_busqueda_aleatoria=10000):

    X_candidatos = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_busqueda_aleatoria, bounds.shape[0]))
    
    ei_values = expected_improvement(X_candidatos, gpr, f_mejor)
    
    idx_max = np.argmax(ei_values)
    
    return X_candidatos[idx_max].reshape(1, -1)

In [63]:
bounds = np.array([[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]])

N_INITIAL = 5   
N_ITERACIONES = 15 
TOTAL_CALLS = N_INITIAL + N_ITERACIONES

print(f"Iniciando Optimización Bayesiana (Total de evaluaciones: {TOTAL_CALLS})\n")


Iniciando Optimización Bayesiana (Total de evaluaciones: 20)



In [64]:
np.random.seed(42) 
X_muestras = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(N_INITIAL, 3))
Y_muestras = funcion_objetivo(X_muestras)

f_mejor = np.min(Y_muestras)
idx_mejor = np.argmin(Y_muestras)
X_mejor = X_muestras[idx_mejor]

print(f"Inicial (f_mejor): {f_mejor:.6f} en {X_mejor}")


Inicial (f_mejor): -3.336411 en [0.83244264 0.21233911 0.18182497]


In [65]:
kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1e2))
gpr = GPR(kernel=kernel, alpha=1e-6, normalize_y=True, n_restarts_optimizer=10, random_state=42)

for i in range(N_ITERACIONES):
    gpr.fit(X_muestras, Y_muestras)
    
    X_siguiente = proximo_punto_optimo(gpr, f_mejor, bounds)
    
    Y_siguiente = funcion_objetivo(X_siguiente)[0]
    
    X_muestras = np.vstack([X_muestras, X_siguiente])
    Y_muestras = np.append(Y_muestras, Y_siguiente)
    
    if Y_siguiente < f_mejor:
        f_mejor = Y_siguiente
        X_mejor = X_siguiente[0]

    print(f"Iteración {i+1:2d}: f(x) = {Y_siguiente:.6f} | Mejor f(x) hasta ahora: {f_mejor:.6f}")
print(f"Mínimo Global Estimado (f(x,y,z)): **{f_mejor:.6f}**")
print(f"Punto Óptimo (x, y, z): **({X_mejor[0]:.4f}, {X_mejor[1]:.4f}, {X_mejor[2]:.4f})**")

Iteración  1: f(x) = -6.561923 | Mejor f(x) hasta ahora: -6.561923
Iteración  2: f(x) = -6.646178 | Mejor f(x) hasta ahora: -6.646178
Iteración  3: f(x) = -2.989764 | Mejor f(x) hasta ahora: -6.646178
Iteración  4: f(x) = -6.210537 | Mejor f(x) hasta ahora: -6.646178
Iteración  5: f(x) = -0.625627 | Mejor f(x) hasta ahora: -6.646178
Iteración  6: f(x) = -2.633143 | Mejor f(x) hasta ahora: -6.646178
Iteración  7: f(x) = -8.754775 | Mejor f(x) hasta ahora: -8.754775
Iteración  8: f(x) = -6.917209 | Mejor f(x) hasta ahora: -8.754775
Iteración  9: f(x) = -7.645515 | Mejor f(x) hasta ahora: -8.754775
Iteración 10: f(x) = -4.617412 | Mejor f(x) hasta ahora: -8.754775
Iteración 11: f(x) = -8.429537 | Mejor f(x) hasta ahora: -8.754775
Iteración 12: f(x) = -4.510327 | Mejor f(x) hasta ahora: -8.754775
Iteración 13: f(x) = -7.729734 | Mejor f(x) hasta ahora: -8.754775
Iteración 14: f(x) = -3.860670 | Mejor f(x) hasta ahora: -8.754775
Iteración 15: f(x) = -5.211143 | Mejor f(x) hasta ahora: -8.75

# Pregunta 2

In [66]:
import pandas as pd
from scipy.stats import norm, t
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

In [67]:
df_productos = pd.read_csv('adidas.csv')

columnas_a_descartar = ['url', 'name', 'sku', 'description', 'images', 'source_website', 
                        'source', 'breadcrumbs', 'language', 'currency', 'color', 'crawled_at']

df_productos = df_productos.drop(columns=columnas_a_descartar)

df_productos['original_price'] = pd.to_numeric(df_productos['original_price'].astype(str).str.replace('$', '', regex=False), errors='coerce')

df_productos['Y_clase_buena'] = (df_productos['average_rating'] >= 4.3).astype(int)

lista_numericas = ['selling_price', 'original_price', 'reviews_count']
lista_categoricas = ['availability', 'category', 'brand', 'country']
df_modelado = df_productos.dropna().reset_index(drop=True)

X_matriz = df_modelado.drop(columns=['average_rating', 'Y_clase_buena'])
Y_vector = df_modelado['Y_clase_buena']

X_matriz = pd.get_dummies(X_matriz, columns=lista_categoricas, drop_first=True)
nombres_factores = X_matriz.columns.tolist()


In [68]:
X_entreno, X_prueba, Y_entreno, Y_prueba = train_test_split(
    X_matriz, Y_vector, train_size=0.7, random_state=42
)
escalador = StandardScaler().fit(X_entreno[lista_numericas])

X_entreno.loc[:, lista_numericas] = escalador.transform(X_entreno[lista_numericas])
X_prueba.loc[:, lista_numericas] = escalador.transform(X_prueba[lista_numericas])

In [69]:
modelo_logistica = LogisticRegression(solver='liblinear', random_state=42)
modelo_logistica.fit(X_entreno, Y_entreno)

Y_prob_prueba = modelo_logistica.predict_proba(X_prueba)[:, 1]
auc_final = roc_auc_score(Y_prueba, Y_prob_prueba)

Y_prob_entreno = modelo_logistica.predict_proba(X_entreno)[:, 1]
auc_entreno = roc_auc_score(Y_entreno, Y_prob_entreno)

In [70]:

N = X_entreno.shape[0]
P = X_entreno.shape[1] 

RSS = np.sum((Y_prob_prueba - Y_prueba) ** 2)

varianza_residual = RSS / (N - P - 1)

beta_intercepto = np.ravel(modelo_logistica.intercept_)
beta_coeficientes = np.ravel(modelo_logistica.coef_)

X_con_intercepto = np.column_stack((np.ones(N), X_entreno.values)).astype(float)

inversa_X_t_X = np.linalg.inv(X_con_intercepto.T @ X_con_intercepto + np.eye(X_con_intercepto.shape[1]) * 1e-6)
matriz_covarianza_beta = inversa_X_t_X * varianza_residual

errores_estandar = np.sqrt(np.diag(matriz_covarianza_beta))

vector_betas = np.concatenate((beta_intercepto, beta_coeficientes))

estadisticos_t = vector_betas / errores_estandar

p_valores = [2 * (1 - t.cdf(np.abs(stat), N - P - 1)) for stat in estadisticos_t]



In [71]:
print(f"Métricas de Clasificación (AUC):")
print(f"  AUC Entrenamiento: {auc_entreno:.4f}")
print(f"  AUC Prueba:        {auc_final:.4f}")

Métricas de Clasificación (AUC):
  AUC Entrenamiento: 0.6750
  AUC Prueba:        0.7643


El AUC de 0.7643 en prueba sugiere que el modelo tiene capacidad de distinguir entre clases, con una precisión de 76.43% sobre el azar.
Todos los factores muestran P-Values muy bajos, indicando que son estadísticamente significativos.

# Pregunta 3

In [72]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.svm import SVC

In [73]:
df = pd.read_csv('diabetes.csv')

In [74]:
y = df['Outcome']
X = df.drop(columns=['Outcome'])
scaler = StandardScaler().fit(X)
X = pd.DataFrame(scaler.transform(X), columns=X.columns)
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496
...,...,...,...,...,...,...,...,...
763,1.827813,-0.622642,0.356432,1.722735,0.870031,0.115169,-0.908682,2.532136
764,-0.547919,0.034598,0.046245,0.405445,-0.692891,0.610154,-0.398282,-0.531023
765,0.342981,0.003301,0.149641,0.154533,0.279594,-0.735190,-0.685193,-0.275760
766,-0.844885,0.159787,-0.470732,-1.288212,-0.692891,-0.240205,-0.371101,1.170732


In [75]:
def run_model(x: np.ndarray, y: np.ndarray, C: float) -> dict:
    kf = KFold(n_splits=10, shuffle=True)

    lr_model = LogisticRegression(max_iter=1000, C=C)
    lr_model.fit(x, y)
    lr_scores = cross_val_score(lr_model, x, y, cv=kf, scoring='precision')
    lr_predict = lr_model.predict_proba(x)[:, 1]
    lr_results = {
        'mean': lr_scores.mean(),
        'std': lr_scores.std(),
        'predictions': lr_predict
    }


    print(
        f"\nLogistic Regression: Precision = {lr_results['mean']:.4f} ± {lr_results['std']:.4f}")

    return {
        'lr_results': lr_results
    }

run_model(X.values, y.values, 2)


Logistic Regression: Precision = 0.7332 ± 0.1258


{'lr_results': {'mean': 0.7331637099284157,
  'std': 0.12577367742882836,
  'predictions': array([0.71981638, 0.04933123, 0.79421539, 0.04226898, 0.90041553,
         0.14753243, 0.06731905, 0.64097778, 0.7097593 , 0.03721347,
         0.22044378, 0.89587106, 0.78299976, 0.63136585, 0.62706761,
         0.39906913, 0.37177202, 0.19736995, 0.35667009, 0.23497608,
         0.39290264, 0.31760973, 0.93916912, 0.29478265, 0.70111706,
         0.44118653, 0.7353875 , 0.04651918, 0.5392435 , 0.27861723,
         0.42608398, 0.57257665, 0.04967327, 0.03712914, 0.43217669,
         0.15065195, 0.66342646, 0.39336248, 0.17298955, 0.5723509 ,
         0.74155082, 0.69469157, 0.11492119, 0.92606855, 0.62715685,
         0.95124578, 0.43321492, 0.04038956, 0.38030842, 0.03940934,
         0.03682523, 0.08594148, 0.06815446, 0.82610491, 0.70780091,
         0.02307931, 0.8817425 , 0.35816004, 0.83150367, 0.18399248,
         0.01011081, 0.52045455, 0.02385955, 0.3085604 , 0.35427064,
         0.119

In [76]:
def run_model(x: np.ndarray, y: np.ndarray) -> dict:
    kf = KFold(n_splits=10, shuffle=True)

    svm_results = {}
    for kernel in kernels:
        svm_model = SVC(kernel=kernel, probability=True, max_iter=1000)
        scores = cross_val_score(svm_model, x, y, cv=kf, scoring='precision')
        svm_results[kernel] = {
            'mean': scores.mean(),
            'std': scores.std()
        }

    lr_model = LogisticRegression(max_iter=1000)
    lr_model.fit(x, y)
    lr_scores = cross_val_score(lr_model, x, y, cv=kf, scoring='precision')
    lr_predict = lr_model.predict_proba(x)[:, 1]
    lr_results = {
        'mean': lr_scores.mean(),
        'std': lr_scores.std(),
        'predictions': lr_predict
    }

    print("SVM Results:")
    for kernel, result in svm_results.items():
        print(f"{kernel}: precision = {result['mean']:.4f} ± {result['std']:.4f}")
    print(
        f"\nLogistic Regression: precision = {lr_results['mean']:.4f} ± {lr_results['std']:.4f}")

    return {
        'svm_results': svm_results,
        'lr_results': lr_results
    }

kernels = ['linear', 'rbf', 'poly']
run_model(X.values, y.values)

SVM Results:
linear: precision = 0.7305 ± 0.0909
rbf: precision = 0.7143 ± 0.1083
poly: precision = 0.7520 ± 0.0957

Logistic Regression: precision = 0.7327 ± 0.1050


{'svm_results': {'linear': {'mean': 0.7304647981585983,
   'std': 0.09093566725853251},
  'rbf': {'mean': 0.7142564635127564, 'std': 0.10834723244016789},
  'poly': {'mean': 0.7519735330029448, 'std': 0.09572202338107426}},
 'lr_results': {'mean': 0.7327120705381575,
  'std': 0.10500195002597894,
  'predictions': array([0.71788557, 0.05004217, 0.7916649 , 0.04292874, 0.89864691,
         0.14842183, 0.06808818, 0.63750014, 0.70997257, 0.03807967,
         0.22122976, 0.89393557, 0.78146623, 0.6337848 , 0.62645407,
         0.39738602, 0.37235677, 0.19810911, 0.35598437, 0.23576997,
         0.39330902, 0.3185104 , 0.93786761, 0.29480507, 0.70010361,
         0.44101334, 0.73329128, 0.04727481, 0.53929577, 0.27939413,
         0.42647917, 0.5716599 , 0.05034582, 0.03780712, 0.43187454,
         0.15193868, 0.66127902, 0.39334987, 0.1735713 , 0.57246524,
         0.73887433, 0.6926571 , 0.11627473, 0.92515896, 0.62521318,
         0.94990929, 0.43202883, 0.04098976, 0.37956448, 0.0398613